<a href="https://colab.research.google.com/github/ffelfis/OrgaDatosTPs/blob/main/TP2/nbFacu_Light_GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lectura de datos de Google Drive

In [1]:
# Lectura de Dataset desde Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Carga de librerías

In [2]:
import pandas as pd
import numpy as np

# Rutas de los archivos a usar

In [3]:
# Ruta train_values.csv
dir_values = '/content/drive/My Drive/75.06 - Organización de Datos/TP1/Data/train_values.csv'

# Ruta train_labels.csv
dir_labels = '/content/drive/My Drive/75.06 - Organización de Datos/TP1/Data/train_labels.csv'

# Ruta test_values.csv
dir_test = '/content/drive/My Drive/75.06 - Organización de Datos/TP1/Data/test_values.csv'

# Ruta submission_format.csv
dir_sub_format = '/content/drive/My Drive/75.06 - Organización de Datos/TP1/Data/submission_format.csv'

# Ruta de Binary Encodings para train_values.csv
dir_Binary_train = '/content/drive/My Drive/75.06 - Organización de Datos/TP2/Binary Encodings/train'

# Ruta de Binary Encodings para test_values.csv
dir_Binary_test = '/content/drive/My Drive/75.06 - Organización de Datos/TP2/Binary Encodings/test'

---
#Entrenamiento
---
### Carga de train

In [4]:
columnas = ['building_id',
 'geo_level_1_id',
 'geo_level_2_id',
 'geo_level_3_id',
 'count_floors_pre_eq',
 'age',
 'area_percentage',
 'height_percentage',
 'has_superstructure_adobe_mud',
 'has_superstructure_mud_mortar_stone',
 'has_superstructure_stone_flag',
 'has_superstructure_cement_mortar_stone',
 'has_superstructure_mud_mortar_brick',
 'has_superstructure_cement_mortar_brick',
 'has_superstructure_timber',
 'has_superstructure_bamboo',
 'has_superstructure_rc_non_engineered',
 'has_superstructure_rc_engineered',
 'has_superstructure_other',
 'count_families',
 'has_secondary_use',
 'has_secondary_use_agriculture',
 'has_secondary_use_hotel',
 'has_secondary_use_rental',
 'has_secondary_use_institution',
 'has_secondary_use_school',
 'has_secondary_use_industry',
 'has_secondary_use_health_post',
 'has_secondary_use_gov_office',
 'has_secondary_use_use_police',
 'has_secondary_use_other']

In [15]:
# Carga de train_values.csv
train = pd.read_csv(dir_values, usecols=columnas)
# Carga de train_labels.csv
labels = pd.read_csv(dir_labels)

### Cambio de tipos de datos

In [6]:
memoria = train.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

Se usan 61.64 MiB


In [16]:
# Columnas numéricas
train['building_id'] = train['building_id'].astype('int32')
train['geo_level_1_id'] = train['geo_level_1_id'].astype('int8')
train['geo_level_2_id'] = train['geo_level_2_id'].astype('int16')
train['geo_level_3_id'] = train['geo_level_3_id'].astype('int16')
train['count_floors_pre_eq'] = train['count_floors_pre_eq'].astype('int8')
train['age'] = train['age'].astype('int16')
train['area_percentage'] = train['area_percentage'].astype('int8')
train['height_percentage'] = train['height_percentage'].astype('int8')
train['count_families'] = train['count_families'].astype('int8')

# Columnas Booleanas
train['has_superstructure_adobe_mud'] = train['has_superstructure_adobe_mud'].astype('bool')
train['has_superstructure_mud_mortar_stone'] = train['has_superstructure_mud_mortar_stone'].astype('bool')
train['has_superstructure_stone_flag'] = train['has_superstructure_stone_flag'].astype('bool')
train['has_superstructure_cement_mortar_stone'] = train['has_superstructure_cement_mortar_stone'].astype('bool')
train['has_superstructure_mud_mortar_brick'] = train['has_superstructure_mud_mortar_brick'].astype('bool')
train['has_superstructure_cement_mortar_brick'] = train['has_superstructure_cement_mortar_brick'].astype('bool')
train['has_superstructure_timber'] = train['has_superstructure_timber'].astype('bool')
train['has_superstructure_bamboo'] = train['has_superstructure_bamboo'].astype('bool')
train['has_superstructure_rc_non_engineered'] = train['has_superstructure_rc_non_engineered'].astype('bool')
train['has_superstructure_rc_engineered'] = train['has_superstructure_rc_engineered'].astype('bool')
train['has_superstructure_other'] = train['has_superstructure_other'].astype('bool')
train['has_secondary_use'] = train['has_secondary_use'].astype('bool')
train['has_secondary_use_agriculture'] = train['has_secondary_use_agriculture'].astype('bool')
train['has_secondary_use_hotel'] = train['has_secondary_use_hotel'].astype('bool')
train['has_secondary_use_rental'] = train['has_secondary_use_rental'].astype('bool')
train['has_secondary_use_institution'] = train['has_secondary_use_institution'].astype('bool')
train['has_secondary_use_school'] = train['has_secondary_use_school'].astype('bool')
train['has_secondary_use_industry'] = train['has_secondary_use_industry'].astype('bool')
train['has_secondary_use_health_post'] = train['has_secondary_use_health_post'].astype('bool')
train['has_secondary_use_gov_office'] = train['has_secondary_use_gov_office'].astype('bool')
train['has_secondary_use_use_police'] = train['has_secondary_use_use_police'].astype('bool')
train['has_secondary_use_other'] = train['has_secondary_use_other'].astype('bool')

In [8]:
memoria = train.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

Se usan 9.2 MiB


### Carga de columnas codificadas: Binary Encoding

In [17]:
# 4 columnas
foundation_type = pd.read_csv(dir_Binary_train+f'/BE_foundation_type.csv', dtype='uint8')
train = train.join(foundation_type)

In [18]:
# 4 columnas
ground_floor_type = pd.read_csv(dir_Binary_train+f'/BE_ground_floor_type.csv', dtype=('uint8'))
train = train.join(ground_floor_type)

In [19]:
# 3 columnas
land_surface_condition = pd.read_csv(dir_Binary_train+f'/BE_land_surface_condition.csv', dtype='uint8')
train = train.join(land_surface_condition)

In [20]:
# 3 columnas
legal_ownership_status = pd.read_csv(dir_Binary_train+f'/BE_legal_ownership_status.csv', dtype='uint8')
train = train.join(legal_ownership_status)

In [21]:
# 3 columnas
other_floor_type = pd.read_csv(dir_Binary_train+f'/BE_other_floor_type.csv', dtype='uint8')
train = train.join(other_floor_type)

In [22]:
# 5 columnas
plan_configuration = pd.read_csv(dir_Binary_train+f'/BE_plan_configuration.csv', dtype='uint8')
train = train.join(plan_configuration)

In [23]:
# 3 columnas
position = pd.read_csv(dir_Binary_train+f'/BE_position.csv', dtype='uint8')
train = train.join(position)

In [24]:
# 3 columnas
roof_type = pd.read_csv(dir_Binary_train+f'/BE_roof_type.csv', dtype='uint8')
train = train.join(roof_type)

In [25]:
memoria = train.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

Se usan 16.15 MiB


### Labels

In [26]:
memoria = labels.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

Se usan 3.98 MiB


In [27]:
labels['building_id'] = labels['building_id'].astype('int32')
labels['damage_grade'] = labels['damage_grade'].astype('int8')

In [28]:
memoria = labels.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

Se usan 1.24 MiB


Así el set de datos es `train` y los labels están en `labels`.



# Modelo Light GBM

In [29]:
# Modelo
import lightgbm
# Para dividir el set de entrenamiento
from sklearn.model_selection import train_test_split

### División de datos para entrenamiento y objetivo

In [30]:
# Set de entrenamiento
X = train
# Labels -- solo la columna de labels
y = labels['damage_grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print('X_train.shape\t', X_train.shape, '\ty_train.shape\t', y_train.shape)
print('X_test.shape\t', X_test.shape, '\ty_test.shape\t', y_test.shape)

X_train.shape	 (195450, 59) 	y_train.shape	 (195450,)
X_test.shape	 (65151, 59) 	y_test.shape	 (65151,)


### Preparación de Datasets
LightGBM necesita leer los datos desde un obejo Dataset especial. Esto se proporciona mediante la función `Dataset()`.

In [ ]:
'''train_data = lightgbm.Dataset(X_train, label=y_train)
test_data = lightgbm.Dataset(X_test, label=y_test)'''

### Párametros de Light GBM

### Instancia de modelo / Entrenamiento

In [70]:
parameters = {
    'learning_rate' : 0.05
}

In [73]:
lgbm = lightgbm.LGBMClassifier(learning_rate=0.05, n_estimators=400)

In [74]:
lgbm.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.05, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=400, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

### Train validation & Test validation

In [34]:
# Métrica de evaluación del modelo
from sklearn.metrics import accuracy_score

In [75]:
train_val = lgbm.predict(X_train)
test_val = lgbm.predict(X_test)

In [76]:
print('Train validation, predicción', np.round(accuracy_score(y_train, train_val),4))
print('Test validation, predicción', np.round(accuracy_score(y_test, test_val),4))

Train validation, predicción 0.7266
Test validation, predicción 0.7154


# Evaluación del predictor

In [59]:
# Para validar
from sklearn.model_selection import KFold
# Para ver progreso inline: limpia la celda en cada iteración
from IPython.display import clear_output

## División de datos para entrenamiento y objetivo

In [60]:
# Set de entrenamiento
X = train
# Labels -- solo la columna de labels
y = labels['damage_grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print('X_train.shape\t', X_train.shape, '\ty_train.shape\t', y_train.shape)
print('X_test.shape\t', X_test.shape, '\ty_test.shape\t', y_test.shape)

X_train.shape	 (195450, 59) 	y_train.shape	 (195450,)
X_test.shape	 (65151, 59) 	y_test.shape	 (65151,)


## Instancias

In [61]:
# n_splits
splits = 10
# Instancia para KFold
kf = KFold(n_splits=splits)

# Listas para guardar los puntajes y errores de la evaluación
accuracy = []

In [77]:
%%time
i = 1

for train_index, test_index in kf.split(train) :
    clear_output(wait=True)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    print('Entrenando fold', i)
    lgbm.fit(X_train, y_train)
    prediction_t = lgbm.predict(X_test)

    accuracy.append(accuracy_score(y_test, prediction_t))
    i = i + 1

Entrenando fold 10
CPU times: user 17min 8s, sys: 3.89 s, total: 17min 12s
Wall time: 8min 49s


In [78]:
precision = np.array(accuracy)

print('Promedio precision', precision.mean().round(4), \
      '\tcon desvío estándar de', precision.std().round(4))

Promedio precision 0.7125 	con desvío estándar de 0.0055


---
# Predicción del Test
---
### Carga Test set

In [ ]:
test = pd.read_csv(dir_test, usecols=columnas)

### Cambio de tipos de datos

In [ ]:
memoria = test.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

In [ ]:
# Columnas numéricas
test['building_id'] = test['building_id'].astype('int32')
test['geo_level_1_id'] = test['geo_level_1_id'].astype('int8')
test['geo_level_2_id'] = test['geo_level_2_id'].astype('int16')
test['geo_level_3_id'] = test['geo_level_3_id'].astype('int16')
test['count_floors_pre_eq'] = test['count_floors_pre_eq'].astype('int8')
test['age'] = test['age'].astype('int16')
test['area_percentage'] = test['area_percentage'].astype('int8')
test['height_percentage'] = test['height_percentage'].astype('int8')
test['count_families'] = test['count_families'].astype('int8')

# Columnas Booleanas
test['has_superstructure_adobe_mud'] = test['has_superstructure_adobe_mud'].astype('bool')
test['has_superstructure_mud_mortar_stone'] = test['has_superstructure_mud_mortar_stone'].astype('bool')
test['has_superstructure_stone_flag'] = test['has_superstructure_stone_flag'].astype('bool')
test['has_superstructure_cement_mortar_stone'] = test['has_superstructure_cement_mortar_stone'].astype('bool')
test['has_superstructure_mud_mortar_brick'] = test['has_superstructure_mud_mortar_brick'].astype('bool')
test['has_superstructure_cement_mortar_brick'] = test['has_superstructure_cement_mortar_brick'].astype('bool')
test['has_superstructure_timber'] = test['has_superstructure_timber'].astype('bool')
test['has_superstructure_bamboo'] = test['has_superstructure_bamboo'].astype('bool')
test['has_superstructure_rc_non_engineered'] = test['has_superstructure_rc_non_engineered'].astype('bool')
test['has_superstructure_rc_engineered'] = test['has_superstructure_rc_engineered'].astype('bool')
test['has_superstructure_other'] = test['has_superstructure_other'].astype('bool')
test['has_secondary_use'] = test['has_secondary_use'].astype('bool')
test['has_secondary_use_agriculture'] = test['has_secondary_use_agriculture'].astype('bool')
test['has_secondary_use_hotel'] = test['has_secondary_use_hotel'].astype('bool')
test['has_secondary_use_rental'] = test['has_secondary_use_rental'].astype('bool')
test['has_secondary_use_institution'] = test['has_secondary_use_institution'].astype('bool')
test['has_secondary_use_school'] = test['has_secondary_use_school'].astype('bool')
test['has_secondary_use_industry'] = test['has_secondary_use_industry'].astype('bool')
test['has_secondary_use_health_post'] = test['has_secondary_use_health_post'].astype('bool')
test['has_secondary_use_gov_office'] = test['has_secondary_use_gov_office'].astype('bool')
test['has_secondary_use_use_police'] = test['has_secondary_use_use_police'].astype('bool')
test['has_secondary_use_other'] = test['has_secondary_use_other'].astype('bool')

In [ ]:
memoria = test.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

### Carga de columnas codificadas: Binary Encoding

In [ ]:
# 4 columnas
foundation_type = pd.read_csv(dir_Binary_test+f'/BE_foundation_type.csv', dtype='uint8')
test = test.join(foundation_type)

In [ ]:
# 4 columnas
ground_floor_type = pd.read_csv(dir_Binary_test+f'/BE_ground_floor_type.csv', dtype=('uint8'))
test = test.join(ground_floor_type)

In [ ]:
# 3 columnas
land_surface_condition = pd.read_csv(dir_Binary_test+f'/BE_land_surface_condition.csv', dtype='uint8')
test = test.join(land_surface_condition)

In [ ]:
# 3 columnas
legal_ownership_status = pd.read_csv(dir_Binary_test+f'/BE_legal_ownership_status.csv', dtype='uint8')
test = test.join(legal_ownership_status)

In [ ]:
# 3 columnas
other_floor_type = pd.read_csv(dir_Binary_test+f'/BE_other_floor_type.csv', dtype='uint8')
test = test.join(other_floor_type)

In [ ]:
# 5 columnas
plan_configuration = pd.read_csv(dir_Binary_test+f'/BE_plan_configuration.csv', dtype='uint8')
test = test.join(plan_configuration)

In [ ]:
# 3 columnas
position = pd.read_csv(dir_Binary_test+f'/BE_position.csv', dtype='uint8')
test = test.join(position)

In [ ]:
# 3 columnas
roof_type = pd.read_csv(dir_Binary_test+f'/BE_roof_type.csv', dtype='uint8')
test = test.join(roof_type)

In [ ]:
memoria = test.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

# Preparando la predicción

In [ ]:
# n_estimators
num_estimadores = 200
# n_jobs
workers = -1
# Instancia para predictor de Random Forest
rfc = RandomForestClassifier(n_estimators=num_estimadores, n_jobs=workers, random_state=42)

In [ ]:
%%time
rfc.fit(X, y)

### Predicción

In [ ]:
prediction = rfc.predict(test)

In [ ]:
prediction

### Submission

In [ ]:
sub_example = pd.read_csv(dir_sub_format)

In [ ]:
sub_example

In [ ]:
submit = pd.DataFrame()
submit['building_id'] = test['building_id']
submit['damage_grade'] = prediction

In [ ]:
submit

In [ ]:
submit.to_csv('submission.csv', index=False)

In [ ]:
aux = pd.read_csv('submission.csv')

In [ ]:
aux